In [86]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from pathlib import Path
from scipy.interpolate import interp1d
from scipy.signal import savgol_filter
from scipy.stats import ttest_ind, ranksums
from decimal import Decimal
import math
import warnings
import statsmodels.api as sm
import os



# Format
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 700)
sns.set(font_scale=1.1)  

# Warnings
pd.options.mode.chained_assignment = None  # default='warn'
warnings.filterwarnings("ignore", category=RuntimeWarning)


from matplotlib.cbook import boxplot_stats  # Para obtener los outliers en un boxplot
# boxplot_stats(df_blocks_filtered['antisaccade_errors']).pop(0)['fliers']


In [ ]:
suj = "antisacadas_127"
df = pd.read_csv(f"./raw_data/{suj}.csv")
df.head(2)

In [119]:
def pre_processing(
    df: pd.DataFrame,
    type_of_saccade: str,
    NEW_SAMPLING_RATE=30,
    baseline_start=-200.0,
    baseline_end=100.0,
    interpolate=False,
    cue_shown_at_left=None,
    threshold_to_accept_sacc=0.5,
    FILTER=1.5,
    filter_by_block=False,
    selected_block=1,
) -> dict:
    """Pre process webcam eye tracking data

    Args:
        df (pd.DataFrame): _description_
        type_of_saccade (str): _description_
        NEW_SAMPLING_RATE (int, optional): _description_. Defaults to 30.
        interpolate (bool, optional): _description_. Defaults to False.
        cue_shown_at_left (_type_, optional): _description_. Defaults to None.
        threshold_to_accept_sacc (float, optional): _description_. Defaults to 0.5.

    Returns:
        dict: _description_
    """

    # 0. Estoy sacando los trials de practica (`isTutorial`) y quedandome con las filas que tengan datos de webgazer
    df_saccade = df.query("not webgazer_data.isnull() and isTutorial == False")

    # Le agrego numero de bloque al df
    blocks = sum([[i] * 20 for i in range(1, 17)], [])
    df_saccade.loc[:, "block"] = blocks

    # Filtro por tipo de sacada
    df_saccade = df_saccade.query("typeOfSaccade == @type_of_saccade")

    # filtro por bloque
    if filter_by_block == True:
        df_saccade = df_saccade.query("block == @selected_block")

    webgazer_data = df_saccade["webgazer_data"].map(eval)

    pro_sacc_errors = 0
    anti_sacc_errors = 0

    ts_xs = []
    pro_sacc_errors_rt = []
    pro_sacc_correct_rt = []
    anti_sacc_errors_rt = []
    anti_sacc_correct_rt = []

    for trial in range(len(webgazer_data)):
        t0 = df_saccade["intraEnd"].iloc[trial]
        tf = df_saccade["fixEnd"].iloc[trial]  # Deberia ser t0 - 200
        xs = np.array([i["x"] for i in webgazer_data.iloc[trial]])
        ts = np.array([i["t"] - t0 for i in webgazer_data.iloc[trial]])

        # Interpolate
        if interpolate:
            f1 = interp1d(ts, xs, kind="linear")
            ts_new = np.linspace(-200, ts[-1], NEW_SAMPLING_RATE)
            xs = f1(ts_new)
            ts = ts_new

        # LOW_FILTER_PIX = 168 # Un 10% para cada lado
        # HIGH_FILTER_PIX = 1512 # Un 10% para cada lado
        # # filtro pasa banda de trials
        # if any(xs > HIGH_FILTER_PIX) or any(xs < LOW_FILTER_PIX):
        #     continue

        # 1. Calcular la mediana entre [baseline_start baseline_end] = Xbase
        x_base = np.median(
            xs[(ts > baseline_start) & (ts < baseline_end)]
        )  # [-200, 100]

        # 2. Calcular la mediana entre [500 700] (se puede antibar) = Xmax (o Xmin segun el signo)
        x_max = np.median(xs[(ts > 500.0) & (ts <= 700.0)])

        # 3. y despues transformas tal que antes de 0 sea 0 y al final sea 1 o -1
        # eso seria: (xs - Xbase) / abs(Xbase - Xmax)
        xs = (xs - x_base) / np.abs(x_base - x_max)

        # 4. Luego de normalizar las estimaciones se espejó la mitad de ellas tal que pudiera asu-
        # mirse que el estímulo visual lateral aparecía siempre del mismo lado. El espejado se realizó
        # multiplicando por −1 las estimaciones de los ensayos en los cuales el estímulo lateral
        # apareciera a izquierda. -> Por lo tanto lo que quede para arriba es mirar en la direccion del estimulo y
        #  para abajo en la direccion opuesta
        if df_saccade["cueShownAtLeft"].iloc[trial] == True:
            xs = xs * -1

        # 5. filtro pasa banda de trials
        if any(xs > FILTER) or any(xs < -FILTER):
            continue

        # # 6. Savitzky-Golay filter
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.savgol_filter.html
        # xs = savgol_filter(xs, 5, 2)

        # 7. Detectar lado en que mira
        xs_after_baseline = xs[ts > baseline_end]
        ts_after_baseline = ts[ts > baseline_end]
        if type_of_saccade == "prosaccade":
            is_sacc_error_in_trial = np.any(
                xs_after_baseline < -threshold_to_accept_sacc
            )
            if is_sacc_error_in_trial:
                pro_sacc_errors += 1
                err_prosacc_idx = np.where(
                    xs_after_baseline < -threshold_to_accept_sacc
                )[0][0]
                err_prosacc_time = ts_after_baseline[err_prosacc_idx]
                pro_sacc_errors_rt.append(f"{err_prosacc_time:.2f}")
            else:
                corr_prosacc_idx = np.where(
                    xs_after_baseline >= threshold_to_accept_sacc
                )[0][0]
                corr_prosacc_time = ts_after_baseline[corr_prosacc_idx]
                pro_sacc_correct_rt.append(f"{corr_prosacc_time:.2f}")

        elif type_of_saccade == "antisaccade":
            is_anti_sacc_error_in_trial = np.any(
                xs_after_baseline > threshold_to_accept_sacc
            )
            if is_anti_sacc_error_in_trial:
                anti_sacc_errors += 1
                err_antisacc_idx = np.where(
                    xs_after_baseline > threshold_to_accept_sacc
                )[0][0]
                err_antisacc_time = ts_after_baseline[err_antisacc_idx]
                anti_sacc_errors_rt.append(f"{err_antisacc_time:.2f}")

            else:
                corr_antisacc_idx = np.where(
                    xs_after_baseline <= -threshold_to_accept_sacc
                )[0][0]
                corr_antisacc_time = ts_after_baseline[corr_antisacc_idx]
                anti_sacc_correct_rt.append(f"{corr_antisacc_time:.2f}")

        ts_xs.append((ts, xs))

    try:
        trials_rejected = 100 - (
            len(ts_xs) / len(webgazer_data) * 100
        )  # len(webgazer_data) = 160 siempre
        pro_saccades_errors_perc = (pro_sacc_errors / len(ts_xs)) * 100
        anti_saccades_errors_perc = (anti_sacc_errors / len(ts_xs)) * 100
    except ZeroDivisionError:
        trials_rejected = 100
        pro_saccades_errors_perc = np.nan
        anti_saccades_errors_perc = np.nan

    return {
        "ts_xs": ts_xs,
        "number_of_trials_remained": len(ts_xs),
        "pro_sacc_errors": pro_sacc_errors,
        "pro_sacc_errors_perc": pro_saccades_errors_perc,
        "anti_sacc_errors": anti_sacc_errors,
        "anti_sacc_errors_perc": anti_saccades_errors_perc,
        "trials_rejected": trials_rejected,
        "pro_sacc_errors_rt": pro_sacc_errors_rt,
        "pro_sacc_correct_rt": pro_sacc_correct_rt,
        "anti_sacc_errors_rt": anti_sacc_errors_rt,
        "anti_sacc_correct_rt": anti_sacc_correct_rt,
        "age": int(eval(df["response"][1])["age"]),
        "px2degree": df[~df["px2deg"].isna()]["px2deg"].iloc[0].round(2),
        "win_width_pixels": df[~df["win_width_deg"].isna()]["win_width_deg"].iloc[0].round(2)
        * df[~df["px2deg"].isna()]["px2deg"].iloc[0].round(2),
        "win_height_pixels": df[~df["win_height_deg"].isna()]["win_height_deg"].iloc[0].round(2)
        * df[~df["px2deg"].isna()]["px2deg"].iloc[0].round(2),
    }


def normalize_in_range(values, a, b):
    """Normalize in range [a,b]"""
    values_normalized = []
    for i in range(len(values)):
        res = (
            (b - a) * ((values[i] - np.min(values))) / (np.max(values) - np.min(values))
        ) + a
        values_normalized.append(res)
    return np.array(values_normalized)


def one_subject(df, suj_number, type_of_saccade):
    fig, axs = plt.subplots(
        3, 1, height_ratios=[1, 4, 1], sharex=True, constrained_layout=True
    )

    suj_number = str(suj_number)
    print("suj_number:", suj_number)
    df = df.query("subject == @suj_number")
    for i in df.query("subject == @suj_number")[type_of_saccade].iloc[0]:
        ts, xs = i[0], i[1]
        axs[1].plot(ts, xs)

        axs[1].axhline(y=0.5, color="k", linestyle="-")
        axs[1].axhline(y=-0.5, color="k", linestyle="-")
        axs[1].axvline(x=0, color="k", linestyle="-")
        axs[1].set_ylabel("x coordinate predictions")
        # axs[1].set_xlabel("time (ms)")
        axs[1].set_xticks(np.arange(-200, 1000, step=100))
        axs[1].set_xlim(-200, 700)
        axs[1].set_ylim(-2, 2)

    if type_of_saccade == "prosaccade":
        data_error = [float(i) for i in df["pro_sacc_errors_rt"].iloc[0]]
        data_correct = [float(i) for i in df["pro_sacc_correct_rt"].iloc[0]]
        sns.kdeplot(data_error, ax=axs[2])
        sns.rugplot(data_error, ax=axs[2], height=0.1)
        sns.kdeplot(data_correct, ax=axs[0])
        sns.rugplot(data_correct, ax=axs[0], height=0.1)
        axs[2].set_xlim([-200, 700])
        axs[2].invert_yaxis()
        axs[0].set_xlim([-200, 700])

    else:
        data_error = [float(i) for i in df["anti_sacc_errors_rt"].iloc[0]]
        data_correct = [float(i) for i in df["anti_sacc_correct_rt"].iloc[0]]
        sns.kdeplot(data_error, ax=axs[0])
        sns.rugplot(data_error, ax=axs[0], height=0.1)
        sns.kdeplot(data_correct, ax=axs[2])
        sns.rugplot(data_correct, ax=axs[2], height=0.1)
        axs[0].set_xlim([-200, 700])
        axs[2].set_xlim([-200, 700])
        axs[2].invert_yaxis()

    plt.suptitle(f"{type_of_saccade}")
    fig.align_ylabels(axs[:])
    fig.supxlabel("time (ms)")
    plt.show()


### Load all files

In [120]:
ALL_FILES_RAW = list(Path("./raw_data").glob("*.csv"))

df_processed_signal = pd.DataFrame(
    {
        "subject": None,
        "prosaccade": None,
        "antisaccade": None,
        "prosaccade_errors": None,
        "antisaccade_errors": None,
    },
    index=[],
)

subjects = []
pro_saccades = []
anti_saccades = []
pro_saccades_errors = []
pro_saccades_errors_perc = []
anti_saccades_errors = []
anti_saccades_errors_perc = []
trials_rejected_prosaccade = []
trials_rejected_antisaccade = []
pro_sacc_errors_rt = []
pro_sacc_correct_rt = []
anti_sacc_errors_rt = []
anti_sacc_correct_rt = []
prosaccade_trials_remained = []
antisaccade_trials_remained = []
age = []
px2degree = []
win_height_pixels = []
win_width_pixels = []



print("processing ...")
for i, suj in enumerate(ALL_FILES_RAW):
    print(suj)
    df = pd.read_csv(suj)
    suj = str(suj).split("/")[-1].split(".")[0].split("_")[-1]
    anti_saccade_dict = pre_processing(
        df, type_of_saccade="antisaccade", interpolate=True
    )
    pro_saccade_dict = pre_processing(
        df, type_of_saccade="prosaccade", interpolate=True
    )
    subjects.append(suj)
    pro_saccades.append(pro_saccade_dict["ts_xs"])
    prosaccade_trials_remained.append(pro_saccade_dict["number_of_trials_remained"])
    antisaccade_trials_remained.append(anti_saccade_dict["number_of_trials_remained"])
    anti_saccades.append(anti_saccade_dict["ts_xs"])
    pro_saccades_errors.append(pro_saccade_dict["pro_sacc_errors"])
    pro_saccades_errors_perc.append(pro_saccade_dict["pro_sacc_errors_perc"])

    anti_saccades_errors.append(anti_saccade_dict["anti_sacc_errors"])
    anti_saccades_errors_perc.append(anti_saccade_dict["anti_sacc_errors_perc"])
    
    trials_rejected_prosaccade.append(pro_saccade_dict["trials_rejected"])

    trials_rejected_antisaccade.append(anti_saccade_dict["trials_rejected"])
    pro_sacc_errors_rt.append(pro_saccade_dict["pro_sacc_errors_rt"])
    pro_sacc_correct_rt.append(pro_saccade_dict["pro_sacc_correct_rt"])
    anti_sacc_errors_rt.append(anti_saccade_dict["anti_sacc_errors_rt"])
    anti_sacc_correct_rt.append(anti_saccade_dict["anti_sacc_correct_rt"])
    age.append(anti_saccade_dict["age"])
    px2degree.append(pro_saccade_dict["px2degree"])
    win_height_pixels.append(pro_saccade_dict["win_height_pixels"])
    win_width_pixels.append(pro_saccade_dict["win_width_pixels"])
    

    # print(" --- ")

df_processed_signal["subject"] = subjects
df_processed_signal["prosaccade"] = pro_saccades
df_processed_signal["antisaccade"] = anti_saccades
df_processed_signal["prosaccade_errors"] = pro_saccades_errors
df_processed_signal["antisaccade_errors"] = anti_saccades_errors
df_processed_signal["delta_errors"] = np.array(anti_saccades_errors) - np.array(
    pro_saccades_errors
)
df_processed_signal[
    "trials_rejected_antisaccade_percentage"
] = trials_rejected_antisaccade
df_processed_signal[
    "trials_rejected_prosaccade_percentage"
] = trials_rejected_prosaccade
df_processed_signal["pro_sacc_errors_rt"] = pro_sacc_errors_rt
df_processed_signal["pro_sacc_correct_rt"] = pro_sacc_correct_rt
df_processed_signal["anti_sacc_errors_rt"] = anti_sacc_errors_rt
df_processed_signal["anti_sacc_correct_rt"] = anti_sacc_correct_rt

# medians
df_processed_signal["pro_sacc_errors_rt_median"] = df_processed_signal[
    "pro_sacc_errors_rt"
].apply(lambda x: np.median([float(i) for i in x]))
df_processed_signal["pro_sacc_correct_rt_median"] = df_processed_signal[
    "pro_sacc_correct_rt"
].apply(lambda x: np.median([float(i) for i in x]))
df_processed_signal["anti_sacc_errors_rt_median"] = df_processed_signal[
    "anti_sacc_errors_rt"
].apply(lambda x: np.median([float(i) for i in x]))
df_processed_signal["anti_sacc_correct_rt_median"] = df_processed_signal[
    "anti_sacc_correct_rt"
].apply(lambda x: np.median([float(i) for i in x]))

# # Percentage of errors
df_processed_signal["prosaccade_errors_perc"] = pro_saccades_errors_perc
df_processed_signal["antisaccade_errors_perc"] = anti_saccades_errors_perc


df_processed_signal["prosaccade_trials_remained"] = prosaccade_trials_remained
df_processed_signal["antisaccade_trials_remained"] = antisaccade_trials_remained

prosaccade_trials_remained

df_processed_signal["age"] = age
df_processed_signal["px2degree"] = px2degree
df_processed_signal["win_width_pixels"] = win_width_pixels
df_processed_signal["win_height_pixels"] = win_height_pixels

print(" ")
print("✅df_processed_signal ready✅")


processing ...
raw_data/antisacadas_99.csv
raw_data/antisacadas_110.csv
raw_data/antisacadas_124.csv
raw_data/antisacadas_115.csv
raw_data/antisacadas_91.csv
raw_data/antisacadas_101.csv
raw_data/antisacadas_104.csv
raw_data/antisacadas_102.csv
raw_data/antisacadas_92.csv
raw_data/antisacadas_107.csv
raw_data/antisacadas_105.csv
raw_data/antisacadas_112.csv
raw_data/antisacadas_98.csv
raw_data/antisacadas_113.csv
raw_data/antisacadas_100.csv
raw_data/antisacadas_118.csv
raw_data/antisacadas_127.csv
raw_data/antisacadas_123.csv
raw_data/antisacadas_109.csv
raw_data/antisacadas_126.csv
raw_data/antisacadas_103.csv
raw_data/antisacadas_114.csv
raw_data/antisacadas_125.csv
raw_data/antisacadas_117.csv
raw_data/antisacadas_111.csv
raw_data/antisacadas_119.csv
 
✅df_processed_signal ready✅


In [ ]:
# # LOW_FILTER_PIX = 200
# # HIGH_FILTER_PIX = 1600

# for TYPE_OF_SACCADE in ['antisaccade', 'prosaccade']:
#     first_point = []
#     range_all = []
#     all_xs = []
#     for SUJ in range(len(df_processed_signal)):
#         ts_one_suj = pd.Series(df_processed_signal[TYPE_OF_SACCADE].iloc[SUJ]).map(lambda x: x[0])
#         xs_one_suj = pd.Series(df_processed_signal[TYPE_OF_SACCADE].iloc[SUJ]).map(lambda x: x[1])
#         for xs in xs_one_suj:
#             # xs = xs[xs > LOW_FILTER_PIX]
#             # xs = xs[xs < HIGH_FILTER_PIX]
#             all_xs.append(xs)

#     all_xs = np.concatenate(all_xs)
#     plt.hist(all_xs)
#     plt.title(F"{TYPE_OF_SACCADE} N={SUJ}")
#     plt.xlim(0, 2000)
#     plt.show()

### Load all files and separate in blocks


In [ ]:
# SEPARACION EN BLOQUES

ALL_FILES_RAW = list(Path("./raw_data").glob("*.csv"))
REJECTED_SUJ = []
all_dfs = []


print("processing ...")
for i, suj in enumerate(ALL_FILES_RAW):
    df = pd.read_csv(suj)
    if suj in REJECTED_SUJ:
        continue
    suj = str(suj).split("/")[-1].split(".")[0].split("_")[-1]

    _df_processed_signal = pd.DataFrame(
        {
            "subject": None,
            "prosaccade": None,
            "antisaccade": None,
            "prosaccade_errors": None,
            "antisaccade_errors": None,
        },
        index=[],
    )

    subjects = []
    prosaccade_trials_remained = []
    antisaccade_trials_remained = []
    pro_saccades = []
    anti_saccades = []
    pro_saccades_errors = []
    pro_saccades_errors_perc = []

    anti_saccades_errors = []
    anti_saccades_errors_perc = []
    trials_rejected_prosaccade = []
    trials_rejected_antisaccade = []
    pro_sacc_errors_rt = []
    pro_sacc_correct_rt = []
    anti_sacc_errors_rt = []
    anti_sacc_correct_rt = []
    age = []
    px2degree = []
    block_number = []

    print(suj)


    for block in range(1, 17):
        anti_saccade_dict = pre_processing(
            df,
            type_of_saccade="antisaccade",
            interpolate=True,
            filter_by_block=True,
            selected_block=block,
        )
        pro_saccade_dict = pre_processing(
            df,
            type_of_saccade="prosaccade",
            interpolate=True,
            filter_by_block=True,
            selected_block=block
        )
        subjects.append(suj)
        pro_saccades.append(pro_saccade_dict["ts_xs"])
        anti_saccades.append(anti_saccade_dict["ts_xs"])

        prosaccade_trials_remained.append(pro_saccade_dict["number_of_trials_remained"])
        antisaccade_trials_remained.append(anti_saccade_dict["number_of_trials_remained"])
    
        pro_saccades_errors.append(pro_saccade_dict["pro_sacc_errors"])
        pro_saccades_errors_perc.append(pro_saccade_dict["pro_sacc_errors_perc"])
        anti_saccades_errors.append(anti_saccade_dict["anti_sacc_errors"])
        anti_saccades_errors_perc.append(anti_saccade_dict["anti_sacc_errors_perc"])
        trials_rejected_prosaccade.append(pro_saccade_dict["trials_rejected"])
        trials_rejected_antisaccade.append(anti_saccade_dict["trials_rejected"])
        pro_sacc_errors_rt.append(pro_saccade_dict["pro_sacc_errors_rt"])
        pro_sacc_correct_rt.append(pro_saccade_dict["pro_sacc_correct_rt"])
        anti_sacc_errors_rt.append(anti_saccade_dict["anti_sacc_errors_rt"])
        anti_sacc_correct_rt.append(anti_saccade_dict["anti_sacc_correct_rt"])
        age.append(anti_saccade_dict["age"])
        px2degree.append(pro_saccade_dict["px2degree"])
        block_number.append(block)
        # print(" --- ")

    _df_processed_signal["subject"] = subjects
    _df_processed_signal["prosaccade"] = pro_saccades
    _df_processed_signal["antisaccade"] = anti_saccades
    _df_processed_signal["prosaccade_errors"] = pro_saccades_errors
    _df_processed_signal["antisaccade_errors"] = anti_saccades_errors
    _df_processed_signal["delta_errors"] = np.array(anti_saccades_errors) - np.array(
        pro_saccades_errors
    )
    _df_processed_signal[
        "trials_rejected_antisaccade_percentage"
    ] = trials_rejected_antisaccade
    _df_processed_signal[
        "trials_rejected_prosaccade_percentage"
    ] = trials_rejected_prosaccade
    _df_processed_signal["pro_sacc_errors_rt"] = pro_sacc_errors_rt
    _df_processed_signal["pro_sacc_correct_rt"] = pro_sacc_correct_rt
    _df_processed_signal["anti_sacc_errors_rt"] = anti_sacc_errors_rt
    _df_processed_signal["anti_sacc_correct_rt"] = anti_sacc_correct_rt

    # median
    _df_processed_signal["pro_sacc_errors_rt_median"] = _df_processed_signal[
        "pro_sacc_errors_rt"
    ].apply(lambda x: np.median([float(i) for i in x]))
    _df_processed_signal["pro_sacc_correct_rt_median"] = _df_processed_signal[
        "pro_sacc_correct_rt"
    ].apply(lambda x: np.median([float(i) for i in x]))
    _df_processed_signal["anti_sacc_errors_rt_median"] = _df_processed_signal[
        "anti_sacc_errors_rt"
    ].apply(lambda x: np.median([float(i) for i in x]))
    _df_processed_signal["anti_sacc_correct_rt_median"] = _df_processed_signal[
        "anti_sacc_correct_rt"
    ].apply(lambda x: np.median([float(i) for i in x]))

    # Percentage of errors
    _df_processed_signal["prosaccade_errors_perc"] = pro_saccades_errors_perc
    _df_processed_signal["antisaccade_errors_perc"] = anti_saccades_errors_perc

    _df_processed_signal["prosaccade_trials_remained"] = prosaccade_trials_remained
    _df_processed_signal["antisaccade_trials_remained"] = antisaccade_trials_remained


    _df_processed_signal["age"] = age
    _df_processed_signal["px2degree"] = px2degree
    _df_processed_signal['block'] = block_number


    all_dfs.append(_df_processed_signal)

print(" ")
print("all_dfs ready✅")

df_all_blocks = pd.concat(all_dfs)

In [27]:
df_all_blocks.query("subject == '99'")['prosaccade'].iloc[0]

[(array([-200.        , -164.89655172, -129.79310345,  -94.68965517,
          -59.5862069 ,  -24.48275862,   10.62068966,   45.72413793,
           80.82758621,  115.93103448,  151.03448276,  186.13793103,
          221.24137931,  256.34482759,  291.44827586,  326.55172414,
          361.65517241,  396.75862069,  431.86206897,  466.96551724,
          502.06896552,  537.17241379,  572.27586207,  607.37931034,
          642.48275862,  677.5862069 ,  712.68965517,  747.79310345,
          782.89655172,  818.        ]),
  array([ 5.08410695e-02,  4.14243459e-02,  2.90448524e-02,  7.58185426e-03,
          3.45379922e-04, -6.55542069e-04, -2.32389611e-02, -3.23276640e-02,
         -9.60882607e-03,  3.61070058e-02,  3.66077470e-02,  2.29166623e-02,
          3.15457303e-02,  5.90777679e-02,  6.21478992e-02,  3.73616100e-02,
          1.58283151e-02,  3.79353326e-02,  4.87299225e-02,  1.87984856e-01,
          4.12626456e-01,  7.15238929e-01,  9.14478347e-01,  1.06788575e+00,
          1.12

In [ ]:
# Constants
MAX_NUMBER_INCORRECTS_PER_BLOCK = (
    14  # Esto implica que en un bloque hizo al menos el 70% mal
)
FIRST_BLOCK = 1
LAST_BLOCK = 16

In [ ]:
# Filter and boxplot anti vs. pro
df_blocks_filtered = (
    df_all_blocks.query(
        "prosaccade_errors < @MAX_NUMBER_INCORRECTS_PER_BLOCK and antisaccade_errors < @MAX_NUMBER_INCORRECTS_PER_BLOCK and @FIRST_BLOCK <= block <= @LAST_BLOCK"
    )  # Con la mitad de los bloques
    .groupby("subject")
    .agg(
        {
            "prosaccade_errors": "sum",
            "antisaccade_errors": "sum",
            "prosaccade_trials_remained": "sum",
            "antisaccade_trials_remained": "sum",
            "pro_sacc_errors_rt": "sum",
            "pro_sacc_correct_rt": "sum",
            "anti_sacc_errors_rt": "sum",
            "anti_sacc_correct_rt": "sum",
            "pro_sacc_errors_rt_median": np.nanmean,
            "anti_sacc_errors_rt_median": np.nanmean,
            "pro_sacc_correct_rt_median": np.nanmean,
            "anti_sacc_correct_rt_median": np.nanmean,
            "block": lambda x: x.nunique(),
        }
    )
)

df_blocks_filtered = df_blocks_filtered.rename({"block": "remained_blocks"}, axis=1)
df_blocks_filtered.insert(2, "delta_errors", df_blocks_filtered['antisaccade_errors'] - df_blocks_filtered['prosaccade_errors'])


# Percentage
df_blocks_filtered["prosaccade_errors_perc"] = (
    df_blocks_filtered["prosaccade_errors"]
    / df_blocks_filtered["prosaccade_trials_remained"]
) * 100


df_blocks_filtered["antisaccade_errors_perc"] = (
    df_blocks_filtered["antisaccade_errors"]
    / df_blocks_filtered["antisaccade_trials_remained"]
) * 100

# Plot
ax = sns.boxplot(
    data=df_blocks_filtered[["prosaccade_errors_perc", "antisaccade_errors_perc"]],
    width=0.3,
)
ax.set_ylabel("%")

boxes = ax.patches
"block"
for i, box in enumerate(boxes):
    box.set_facecolor("w")


print(
    ranksums(
        df_blocks_filtered["prosaccade_errors_perc"],
        df_blocks_filtered["antisaccade_errors_perc"],
    )
)
print(f"N={len(df_blocks_filtered)}, blocks={df_blocks_filtered['remained_blocks'].max()}")

print(f"Cantidad de bloques filtrados: {(len(df_blocks_filtered['remained_blocks']) * 16)  - df_blocks_filtered['remained_blocks'].sum()}")
df_blocks_filtered.describe()

In [ ]:
# Testeando normalidad
pro_data = df_blocks_filtered["prosaccade_errors_perc"],
anti_data = df_blocks_filtered["antisaccade_errors_perc"],
sm.qqplot(pro_data[0], line='45', fit = True)
sm.qqplot(anti_data[0], line='45', fit = True)
plt.show()

In [ ]:
# % Incorrectos por bloque

all_dfs = []
for last_block in range(1, 17):
    # filtrar bloques

    FIRST_BLOCK = 1
    LAST_BLOCK = last_block

    df_blocks_filtered = (
        df_all_blocks.query(
            "prosaccade_errors < @MAX_NUMBER_INCORRECTS_PER_BLOCK and antisaccade_errors < @MAX_NUMBER_INCORRECTS_PER_BLOCK and @FIRST_BLOCK <= block <= @LAST_BLOCK"
        )  # Con la mitad de los bloques
        .groupby("subject")
        .agg(
            {
                "prosaccade_errors": "sum",
                "antisaccade_errors": "sum",
                "prosaccade_trials_remained": "sum",
                "antisaccade_trials_remained": "sum",
                "pro_sacc_errors_rt": "sum",
                "pro_sacc_correct_rt": "sum",
                "anti_sacc_errors_rt": "sum",
                "anti_sacc_correct_rt": "sum",
                "pro_sacc_errors_rt_median": np.nanmean,
                "anti_sacc_errors_rt_median": np.nanmean,
                "pro_sacc_correct_rt_median": np.nanmean,
                "anti_sacc_correct_rt_median": np.nanmean,
                "block": lambda x: x.nunique(),
            }
        )
    )

    df_blocks_filtered = df_blocks_filtered.rename({"block": "remained_blocks"}, axis=1)


    # Percentage
    df_blocks_filtered["prosaccade_errors_perc"] = (
        df_blocks_filtered["prosaccade_errors"]
        / df_blocks_filtered["prosaccade_trials_remained"]
    ) * 100


    df_blocks_filtered["antisaccade_errors_perc"] = (
        df_blocks_filtered["antisaccade_errors"]
        / df_blocks_filtered["antisaccade_trials_remained"]
    ) * 100
    all_dfs.append(df_blocks_filtered)

all_dataframes = pd.concat(all_dfs)

all_dataframes

# Plot
ax = sns.boxplot(
    data=all_dataframes.melt(id_vars=['remained_blocks'], value_vars=['prosaccade_errors_perc', 'antisaccade_errors_perc'],
                 var_name='Type of trial', value_name='% of incorrect trials'),
    x='remained_blocks',
    y='% of incorrect trials',
    hue='Type of trial',
    showfliers = True
)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))
plt.show()

ax = sns.boxplot(
    data=all_dataframes.melt(id_vars=['remained_blocks'], value_vars=['prosaccade_errors_perc', 'antisaccade_errors_perc'],
                 var_name='Type of trial', value_name='% of incorrect trials'),
    x='remained_blocks',
    y='% of incorrect trials',
    hue='Type of trial',
    showfliers = False
)
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))


plt.show()

In [ ]:
# RT VERSION CON BLOQUES

# Boxplot with means
ax = sns.boxplot(
    data=df_blocks_filtered[
        [
            "pro_sacc_errors_rt_median",
            "anti_sacc_errors_rt_median",
            "pro_sacc_correct_rt_median",
            "anti_sacc_correct_rt_median",
        ]
    ],
    width=0.3,
)

boxes = ax.patches
"block"
for i, box in enumerate(boxes):
    box.set_facecolor("w")

plt.ylabel("time (ms)")
ax.set_xticklabels(
    [
        "Prosaccade\nincorrect\ntrials\nRT",
        "Antisaccade\nincorrect\ntrials\nRT",
        "Prosaccade\ncorrect\ntrials\nRT",
        "Antisaccade\ncorrect\ntrials\nRT",
    ]
)

plt.setp(ax.artists, edgecolor="k", facecolor="w")
plt.setp(ax.lines, color="k")

plt.show()

print(
    f"""RT condition [pro - anti] x result [correct - incorrect]
ranksums pro_sacc_errors_rt_median vs. anti_sacc_errors_rt_median:  {Decimal(ranksums(df_blocks_filtered['pro_sacc_errors_rt_median'],df_blocks_filtered['anti_sacc_errors_rt_median'], nan_policy='omit')[1]):.5f}
ranksums pro_sacc_correct_rt_median vs. anti_sacc_correct_rt_median: {Decimal(ranksums(df_blocks_filtered['pro_sacc_correct_rt_median'],df_blocks_filtered['anti_sacc_correct_rt_median'], nan_policy='omit')[1]):.5f}
"""
)

df_blocks_filtered.describe()

### PLOT EYE DATA

In [ ]:
ALL_FILES_RAW = list(Path("./raw_data").glob("*.csv"))

# Plot all subjects
for file in enumerate(ALL_FILES_RAW):
    print("---")
    suj = str(file).split("/")[-1].split(".")[0].split("_")[-1]
    print(suj)
    one_subject(df_processed_signal, suj, "prosaccade")
    one_subject(df_processed_signal, suj, "antisaccade")

In [ ]:
for i in ALL_FILES_RAW:
    print(i)

# HASTA ACA ESTUVE MODIFICANDO, FALTA ADAPTAR CON BLOQUES


In [ ]:
# HASTA ACA ESTUVE MODIFICANDO, FALTA ADAPTAR CON BLOQUES

df_accepted_trials_perc = 100  - df_processed_signal[[
        "trials_rejected_prosaccade_percentage",
        "trials_rejected_antisaccade_percentage",
    ]]

df_accepted_trials_perc.columns = ['trials_accepted_prosaccade_perc', 'trials_accepted_antisaccade_perc']

df_accepted_trials_perc

ax = sns.boxplot(
    data=df_accepted_trials_perc,
    width=0.3,
)

# # Select which box you want to change
boxes = ax.patches

for i, box in enumerate(boxes):
    box.set_facecolor("w")
print(
    f"""Percentage of accepted trials
N={len(df_accepted_trials_perc)}
ranksums p={(ranksums(df_accepted_trials_perc['trials_accepted_prosaccade_perc'], df_accepted_trials_perc['trials_accepted_antisaccade_perc'])[-1]) :.3f}"""
)

ax.set_xticklabels(
    [
        "Prosaccade\naccepted trials",
        "Antisaccade\naccepted trials",
    ]
)
plt.ylabel('%')
plt.show()

df_accepted_trials_perc.describe()

In [ ]:
df_processed_signal.boxplot(
    column=[
        "trials_rejected_prosaccade_percentage",
        "trials_rejected_antisaccade_percentage",
    ],
    fontsize=15,
    rot=90,
)
print(df_processed_signal.shape)
plt.show()

df_processed_signal[
    [
        "trials_rejected_prosaccade_percentage",
        "trials_rejected_antisaccade_percentage",
    ]
].describe()

print(
    ranksums(
        df_processed_signal["trials_rejected_prosaccade_percentage"],
        df_processed_signal["trials_rejected_antisaccade_percentage"],
    )
)

In [ ]:
# Plot all subjects
for file in enumerate(ALL_FILES_RAW):
    print("---")
    suj = str(file).split("/")[-1].split(".")[0].split("_")[-1]
    print(suj)
    one_subject(df_processed_signal, suj, "prosaccade")
    one_subject(df_processed_signal, suj, "antisaccade")

### Saccade detection


### Preparing test dataset

`infile`: Data file with eye gaze recordings to process. <br>
The first two columns in this file must contain x and y coordinates, while each line is a timepoint (no header). <br>
The file is read with NumPy's recfromcsv and may be compressed. The columns are expected to be seperated by tabulators (\t).

In [84]:
df_processed_signal.head(2)

,subject,prosaccade,antisaccade,prosaccade_errors,antisaccade_errors,delta_errors,trials_rejected_antisaccade_percentage,trials_rejected_prosaccade_percentage,pro_sacc_errors_rt,pro_sacc_correct_rt,anti_sacc_errors_rt,anti_sacc_correct_rt,pro_sacc_errors_rt_median,pro_sacc_correct_rt_median,anti_sacc_errors_rt_median,anti_sacc_correct_rt_median,prosaccade_errors_perc,antisaccade_errors_perc,prosaccade_trials_remained,antisaccade_trials_remained,age,px2degree,win_width_pixels,win_height_pixels
0,99,"[([-200.0, -164.89655172413794, -129.793103448...","[([-200.0, -165.0689655172414, -130.1379310344...",5,10,5,12.50,8.75,"[373.90, 275.03, 322.93, 574.55, 170.59]","[537.17, 526.31, 388.55, 436.21, 444.28, 404.9...","[356.14, 430.00, 297.72, 400.28, 404.97, 402.6...","[428.76, 446.14, 499.31, 487.59, 394.41, 542.2...",322.930,405.55,386.5,450.59,3.424658,7.142857,146,140,35,29.22,1680.1500,861.9900
1,110,"[([-200.0, -164.79310344827587, -129.586206896...","[([-200.0, -166.27586206896552, -132.551724137...",8,6,-2,16.25,8.75,"[150.00, 219.59, 331.21, 784.83, 353.38, 386.2...","[433.72, 470.24, 434.97, 445.52, 461.07, 416.9...","[396.17, 346.21, 119.03, 455.83, 541.93, 529.21]","[508.21, 397.93, 415.10, 507.48, 440.55, 442.7...",369.795,427.83,426.0,453.21,5.479452,4.477612,146,134,33,38.38,1680.2764,862.0148


In [127]:
SUJ = 1
rows = []
for i, data in enumerate(df_processed_signal.iloc[SUJ]["prosaccade"]):

    # create a new row as a dictionary
    new_row = {"x": data[1], "y": np.zeros(len(data[1])), "t": data[0]}
    rows.append(new_row)
    # append the new row to the dataframe
    # df_sacc_detect_test = pd.concat([df_sacc_detect_test, pd.DataFrame.from_records(new_row)], ignore_index=True)

df_sacc_detect_test = pd.DataFrame(rows)
df_sacc_detect_test_expl = df_sacc_detect_test.explode(
    ["x", "y", "t"], ignore_index=True
)

df_sacc_detect_test_expl = df_sacc_detect_test_expl[['x', 'y']]

# Datos de un sujeto
df_sacc_detect_test_expl.iloc[:30].to_csv("detect_sacc.csv", sep="\t", header=False, index=False)

In [131]:

# def fixations_saccades_detection(raw, et_channels_meg, subject, sac_max_vel=1500, fix_max_amp=1.5,
#                                  screen_size=38, screen_resolution=1920, force_run=False):

OUT_FNAME = f"Fix_Sac_detection.tsv"
OUT_FOLDER = '/'

# Remodnav parameters
FNAME = "detect_sacc.csv"
PX2DEG = df_processed_signal['px2degree'][SUJ]
SFREQ = 30 # Sampling rate after inteporlation
FORCE_RUN = True

# if not force_run:
#     try:
#         # Load pre run saccades and fixation detection
#         sac_fix = pd.read_csv(out_folder + out_fname, sep="\t")
#         print("\nSaccades and fixations loaded")
#     except:
#         force_run = True

if FORCE_RUN:
    # If not pre run data, run
    print("\nRunning saccades and fixations detection")

    # Run Remodnav not considering pursuit class and min fixations 100 ms
    command = (
        f"remodnav {FNAME} {OUT_FNAME} {PX2DEG} {SFREQ} --savgol-length {0.1} --min-pursuit-duration {2} "
        f"--max-pso-duration {0.0} --min-fixation-duration {0.05} --max-vel {5000}"
    )
    os.system(command)

    # Read results file with detections
    df_sac_fix = pd.read_csv(OUT_FNAME, sep="\t")



Running saccades and fixations detection


In [132]:
df_sac_fix

,onset,duration,label,start_x,start_y,end_x,end_y,amp,peak_vel,med_vel,avg_vel
0,0.000,0.533,FIXA,0.0,0.0,0.1,0.0,4.837,76.446,8.335,19.245
1,0.533,0.233,SACC,0.1,0.0,1.0,0.0,33.730,425.438,78.674,145.568
2,0.767,0.233,FIXA,1.0,0.0,0.9,0.0,4.766,51.686,7.641,20.424


### Other

In [ ]:
# Podemos poner 0 - 150
# Graficar solo los eerrores

# Calcular para cada trial correcto o incorrecto y el tiempo (y promedios +- desvio)
# Tablita cmo en la tesis (2 porcentajes y 4 tiempos)


In [ ]:
# Tiempos en el experimento (ms)

# Inter-trial time (ITI) = 925
# Prensentacion de la cue de tipo de tarea =  getRandomIntInclusive(900, 1500) -> es variable!
# Presentacion del cuadrado central que es igual para todos (cue sin informacion) = 200
# tiempo durante el que aparece la visualCue = 150
# Tiempo para responder (responseAwait) = 650

925 + 1500 + 200 + 150 + 650


In [ ]:
df_processed_signal_comun

In [ ]:
df_processed_signal

In [ ]:
plt.scatter(df_processed_signal['trials_rejected_antisaccade_percentage'], df_processed_signal_comun['trials_rejected_antisaccade_percentage'])
plt.xlabel('pixels')
plt.ylabel('[-1.5, 1.5]')
plt.show()

In [ ]:
df_processed_signal['trials_rejected_antisaccade_percentage'] - df_processed_signal_comun['trials_rejected_antisaccade_percentage']